In [ ]:
! pip install html5lib


In [2]:
from tqdm import tqdm 

def selection(entries: list[str]):
    mods = []
    for entry in tqdm(entries, total=len(entries)):
        if len(entry.strip()) > 50:
            continue 
        if "/" in entry:
            mods.append(entry.split("/")[0].strip())
        else: 
            mods.append(entry.strip())
    
    mods = list(set(mods))
    return mods

In [3]:
import requests
from bs4 import BeautifulSoup

url = "https://www.myplan.com/careers/browse-alphabetically.html?letter=all&sort=titles&page=all"
r = requests.get(url)

soup = BeautifulSoup(r.content, 'html5lib')

In [4]:
careers = soup.find('td', attrs={'class': 'box_table'}).findAll('a', attrs={'class': 'list-link'})

In [5]:
records = {career.text: "https://www.myplan.com"+career['href'] for career in careers}

In [6]:
sample = records['Actors']

In [ ]:
sample.split("?")[0]

In [9]:
from requests.exceptions import SSLError

tries = 0
while tries != -1:
    try:
        response = requests.get(sample.split("?")[0].replace("summary", "description"))
        tries = -1
    except SSLError as e:
        tries += 1

In [10]:
soup1 = BeautifulSoup(response.content, 'html5lib')

In [ ]:

sample.split("?")[0].replace("summary", "description")

In [ ]:
for k in soup1.findAll("span"):
    print(k.text)

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# ! pip install selenium
! pip install webdriver_manager


In [15]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(options=options)

In [16]:
driver.get(sample.split("?")[0].replace("summary", "description"))
element = driver.find_element(By.XPATH, "/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center/table/tbody/tr[1]/td[3]/table/tbody/tr[4]/td/table/tbody/tr[5]")

In [ ]:
print(element.text)

In [ ]:
from tqdm import tqdm
from selenium.webdriver.common.by import By

count = 0
for entry, hyperlink in tqdm(records.items(), total=len(records)):
    count += 1
    if count < 100:
        continue  # Skipping first 100 entries

    entry = entry.replace("/", " or ")  # Avoid issues with file names
    result = ""
    xpath = "/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center/table/tbody/tr[1]/td[3]/table/tbody/tr[4]"

    # Scrape description page
    driver.get(hyperlink.split("?")[0].replace("summary", "description"))
    element = driver.find_element(By.XPATH, xpath)
    result += element.text + "\n\n"

    # Scrape requirements page
    driver.get(hyperlink.split("?")[0].replace("summary", "requirements"))
    element = driver.find_element(By.XPATH, xpath)
    result += element.text  

    # ✅ FIX: Save the file with UTF-8 encoding
    with open(f"../data/scraped/{entry}.txt", "w", encoding="utf-8") as file:
        file.write(result)
